# Knowledge-based Recommender System (KBRS)

Questo notebook guida attraverso le prime fasi di pre-elaborazione e arricchimento dei dati per un sistema di raccomandazione basato sulla conoscenza. Il processo si articola in tre passaggi principali:

1.  **Caricamento Dati**: Carica i file `.csv` grezzi in DataFrame di Pandas.
2.  **Preparazione Catalogo**: Unisce i dati dei film e i link per creare un catalogo unico di film.
3.  **Arricchimento Dati**: Arricchisce il catalogo film con informazioni da Wikidata e DBpedia.
4.  **Unione Dati Finali**: Unisce le valutazioni degli utenti con il catalogo arricchito.

## 1. Caricamento dei Dati
Iniziamo caricando i dataset `movies.csv`, `links.csv` e `ratings.csv` utilizzando le funzioni dal modulo `src.data_manager`.

In [1]:
import sys
import os
from IPython.display import display

# Aggiusta il percorso per l'import del modulo `src`
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.data_manager import load_data, join_dataframes, enrich_movies

# Carica i dati grezzi dai file CSV
ratings_df, movies_df, links_df = load_data()

if movies_df is not None and links_df is not None and ratings_df is not None:
    print("\n--- Dati caricati ---")
    
    print("movies.csv:", movies_df.shape)
    print("\nlinks.csv:", links_df.shape)
    print("\nratings.csv:", ratings_df.shape)
    display(movies_df.head())
    display(links_df.head())
    display(ratings_df.head())
    

Dati caricati con successo.

--- Dati caricati ---
movies.csv: (9742, 3)

links.csv: (9742, 3)

ratings.csv: (100836, 4)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## 2. Preparazione del Catalogo Film Unico
Uniamo i DataFrame `movies` e `links` per creare un catalogo di film unici. Questo passaggio è cruciale per evitare chiamate API duplicate e rendere il processo di arricchimento efficiente.

In [2]:
unique_movie_catalog = join_dataframes(movies_df, links_df)
unique_movie_catalog = unique_movie_catalog.drop(columns=['tmdbId'])
unique_movie_catalog['imdbId'] = unique_movie_catalog['imdbId'].apply(lambda x: f"tt{int(x):07d}")

print("Catalogo film unico creato.")
print("Shape del catalogo unico:", unique_movie_catalog.shape)
display(unique_movie_catalog.head())

Catalogo film unico creato.
Shape del catalogo unico: (9742, 4)


,movieId,title,genres,imdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,tt0114709
1,2,Jumanji (1995),Adventure|Children|Fantasy,tt0113497
2,3,Grumpier Old Men (1995),Comedy|Romance,tt0113228
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,tt0114885
4,5,Father of the Bride Part II (1995),Comedy,tt0113041


## 3. Arricchimento del Catalogo Film
Ora arricchiamo il catalogo unico di film con i dati semantici di Wikidata e DBpedia. La funzione `enrich_movies` gestirà le chiamate API in batch e mostrerà l'avanzamento con una barra di caricamento.

In [3]:
enriched_movie_catalog = enrich_movies(unique_movie_catalog)

print("\n--- Dati arricchiti ---")
print("Shape del catalogo arricchito:", enriched_movie_catalog.shape)
display(enriched_movie_catalog.head())

Nessun file di ripristino trovato. Avvio l'elaborazione dall'inizio.


Arricchimento dati film:   6%|▋         | 625/9742 [00:46<10:37, 14.31it/s]

Errore nella query Wikidata: HTTP Error 429: Too Many Requests


Arricchimento dati film:   7%|▋         | 675/9742 [00:49<09:46, 15.46it/s]

Errore nella query Wikidata: HTTP Error 429: Too Many Requests


...

Errore nella query Wikidata: HTTP Error 429: Too Many Requests


Arricchimento dati film:  16%|█▌        | 1525/9742 [01:49<08:53, 15.40it/s]

Errore nella query Wikidata: HTTP Error 429: Too Many Requests


Arricchimento dati film:  46%|████▌     | 4450/9742 [05:18<06:09, 14.34it/s]

KeyboardInterrupt: 

## 4. Unione finale con le Valutazioni
Infine, uniamo il catalogo di film arricchito con le valutazioni degli utenti. Questo DataFrame finale contiene tutte le informazioni necessarie per la successiva fase di implementazione del KBRS.

In [ ]:
if 'enriched_movie_catalog' in locals() and ratings_df is not None:
    final_data = join_dataframes(ratings_df, enriched_movie_catalog)

    print("\n--- DataFrame finale unito ---")
    print("Shape del DataFrame finale:", final_data.shape)
    display(final_data.head())